In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime as dt
import yfinance as yf
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import statsmodels.api as sm


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [7]:
from fredapi import Fred
fred = Fred(api_key = 'ec682064ba501918755568565884325e')
start_date = ('01/01/1990')
end_date = ('12/31/2017')

eco_unc = pd.Series((fred.get_series('WLEMUINDXD', start_date, end_date)), name = 'eco_unc')
tbill = pd.Series((fred.get_series('DTB3', start_date, end_date)), name = 'tbill')
wti = pd.Series((fred.get_series('DCOILWTICO', start_date, end_date)), name = 'wti')
vix = pd.Series((fred.get_series('VIXCLS', start_date, end_date)), name = 'vix')
# fred continuous unemployment claims
cont_claims = pd.Series((fred.get_series('CCSA', start_date, end_date)), name = 'cont_claims')

# effr = pd.Series((fred.get_series('EFFR', start_date, end_date)), name = 'effr')

In [19]:
import datetime as dt

cont_claims
cclaims = pd.Series()
for i in cont_claims:
    counter = 1
    if i == 0:
        cclaims.append(cont_claims[i])
    elif counter < 6:
        # increase datetime delta by counter
        cclaims.index[i + counter] = cont_claims.index[i] + dt.timedelta(days = counter)
        cclaims[i + counter] = cont_claims[i]




<ipython-input-19-e1e8c2c143fa>:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cclaims = pd.Series()
<ipython-input-19-e1e8c2c143fa>:11: FutureWarning: Indexing with a float is deprecated, and will raise an IndexError in pandas 2.0. You can manually convert to an integer key instead.
  cclaims.index[i + counter] = cont_claims.index[i] + dt.timedelta(days = counter)


IndexError: index 2347000 is out of bounds for axis 0 with size 1461

In [22]:
cont_claims.index[0] + dt.timedelta(days = 2)

Timestamp('1990-01-08 00:00:00')

In [5]:
data = pd.concat([eco_unc, tbill, wti, vix], axis =1).dropna()

In [6]:
st_date = dt.datetime(2000, 1, 1)
ed_date = dt.datetime(2017, 12, 31)
SNP500 = yf.download('^GSPC', st_date, ed_date)
SNP500 = SNP500['Adj Close'].to_frame()
SNP500.fillna(method = 'bfill', inplace=True)
SNP500['SMA 5'] = SNP500['Adj Close'].rolling(5).mean()
SNP500['SMA 200'] = SNP500['Adj Close'].rolling(200).mean()
conditions = [SNP500['SMA 5'] >= SNP500['SMA 200'],
              SNP500['SMA 5'] < SNP500['SMA 200']]
choices = [1, 0]
SNP500['outcome'] = np.select(conditions, choices, 0)
SNP500 = SNP500['outcome'].dropna()


[*********************100%***********************]  1 of 1 completed


In [7]:
features = pd.merge(data, SNP500, left_index = True, right_index = True).dropna()
X = features[['eco_unc', 'tbill', 'wti', 'vix', 'cont_claims']] 
y = features['outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=101)

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [8]:
features

,eco_unc,tbill,wti,vix,cont_claims,outcome


In [ ]:
data.corr()

,eco_unc,tbill,wti,vix
eco_unc,1.000000,0.040307,0.004497,0.360580
tbill,0.040307,1.000000,0.114596,-0.039390
wti,0.004497,0.114596,1.000000,-0.100543
vix,0.360580,-0.039390,-0.100543,1.000000


In [ ]:
vif = pd.DataFrame()
vif["ft"] = data.columns
vif["VIF"] = [variance_inflation_factor(data.values, i)
                         for i in range(len(data.columns))]
vif

,ft,VIF
0,eco_unc,1.640543
1,tbill,1.384953
2,wti,4.557716
3,vix,5.012340


In [ ]:
log_regression = LogisticRegression()
model = log_regression.fit(X_train,y_train)

y_pred = log_regression.predict(X_test)

print('score', log_regression.score(X_test, y_test))   #Return the mean accuracy on the given test data and labels.

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

score 0.7757936507936508
              precision    recall  f1-score   support

           0       0.80      0.49      0.61      1426
           1       0.77      0.93      0.84      2606

    accuracy                           0.78      4032
   macro avg       0.79      0.71      0.72      4032
weighted avg       0.78      0.78      0.76      4032



In [ ]:
model.coef_

array([[-0.01106758, -0.2050257 ,  0.01015023]])

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[ 694,  732],
       [ 172, 2434]])

In [ ]:
model1 = sm.Logit(y_train, X_train)
result = model1.fit()
result.params

Optimization terminated successfully.
         Current function value: 0.459164
         Iterations 6


eco_unc   -0.001794
tbill     -1.326107
wti        0.046885
vix       -0.091747
dtype: float64

In [ ]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                outcome   No. Observations:                 1761
Model:                          Logit   Df Residuals:                     1757
Method:                           MLE   Df Model:                            3
Date:                Tue, 15 Feb 2022   Pseudo R-squ.:                  0.1947
Time:                        22:31:59   Log-Likelihood:                -808.59
converged:                       True   LL-Null:                       -1004.0
Covariance Type:            nonrobust   LLR p-value:                 2.078e-84
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
eco_unc       -0.0018      0.001     -2.191      0.028      -0.003      -0.000
tbill         -1.3261      0.115    -11.520      0.000      -1.552      -1.100
wti            0.0469      0.002     18.929      0.000       0.042       0.052
vix           -0.0917      0.007    -12.374      0.000      -0.106      -0.077
==============================================================================
"""